In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

In [12]:
# 1. Préparation des données
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

In [ ]:
# 2. Modèles
class FashionMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),              # [batch,1,28,28] -> [batch,784]
            nn.Linear(28*28, 512),     # 784 -> 512
            nn.ReLU(),                 # activation
            nn.Dropout(0.2),           # régularisation
            nn.Linear(512, 256),       # 512 -> 256
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 128),       # 256 -> 128
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 10)         # 128 -> 10 (10 classes)
        )
    def forward(self, x):
        return self.model(x)            # sortie : 10 scores (logits)

class FashionCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), # image 1x28x28 -> 32x28x28
            nn.ReLU(),
            nn.MaxPool2d(2),                # 32x28x28 -> 32x14x14
            nn.Conv2d(32, 64, 3, padding=1),# 32x14x14 -> 64x14x14
            nn.ReLU(),
            nn.MaxPool2d(2)                 # 64x14x14 -> 64x7x7
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),         # 3136 -> 128
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 10)              # 128 -> 10 (10 classes)
        )
    def forward(self, x):
        x = self.conv(x)
        return self.fc(x)                   # sortie : 10 scores (logits)

In [14]:
# 2.5. TensorBoard writer
writer = SummaryWriter("runs/fashionmnist_experiment")

In [15]:
# 3. Entraînement
def train(model, loader, optimizer, loss_fn, device, epoch):
    model.train()
    running_loss = 0.0
    for batch_idx, (X, y) in enumerate(loader):
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if batch_idx % 100 == 0:
            n_iter = epoch * len(loader) + batch_idx
            writer.add_scalar("Loss/train", loss.item(), n_iter)
    return running_loss / len(loader)

def test(model, loader, device, epoch):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            correct += (pred.argmax(1) == y).sum().item()
            total += y.size(0)
    acc = correct / total
    writer.add_scalar("Accuracy/test", acc, epoch)
    return acc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
# MLP
mlp = FashionMLP().to(device)
optimizer_mlp = optim.Adam(mlp.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
epochs = 10
for epoch in range(epochs):
    train(mlp, train_loader, optimizer_mlp, loss_fn, device, epoch)
    acc = test(mlp, test_loader, device, epoch)
    print(f"MLP Epoch {epoch+1}: Test accuracy = {acc:.4f}")
    if epoch == 0 or epoch == epochs-1:
        mlp.eval()
        images, labels = next(iter(test_loader))
        images = images.to(device)
        outputs = mlp(images)
        preds = outputs.argmax(1)
        img_grid = utils.make_grid(images[:16].cpu(), nrow=4, normalize=True)
        writer.add_image('FashionMNIST MLP Images', img_grid, epoch)
        writer.add_text('MLP Predictions', str(preds[:16].cpu().numpy()), epoch)
        writer.add_text('MLP Labels', str(labels[:16].cpu().numpy()), epoch)

MLP Epoch 1: Test accuracy = 0.8386
MLP Epoch 2: Test accuracy = 0.8550
MLP Epoch 3: Test accuracy = 0.8581
MLP Epoch 4: Test accuracy = 0.8661
MLP Epoch 5: Test accuracy = 0.8735
MLP Epoch 6: Test accuracy = 0.8737
MLP Epoch 7: Test accuracy = 0.8760
MLP Epoch 8: Test accuracy = 0.8769
MLP Epoch 9: Test accuracy = 0.8798
MLP Epoch 10: Test accuracy = 0.8810


In [19]:
# CNN
cnn = FashionCNN().to(device)
optimizer_cnn = optim.Adam(cnn.parameters(), lr=1e-3)
for epoch in range(epochs):
    train(cnn, train_loader, optimizer_cnn, loss_fn, device, epoch)
    acc = test(cnn, test_loader, device, epoch)
    print(f"CNN Epoch {epoch+1}: Test accuracy = {acc:.4f}")
    if epoch == 0 or epoch == epochs-1:
        cnn.eval()
        images, labels = next(iter(test_loader))
        images = images.to(device)
        outputs = cnn(images)
        preds = outputs.argmax(1)
        img_grid = utils.make_grid(images[:16].cpu(), nrow=4, normalize=True)
        writer.add_image('FashionMNIST CNN Images', img_grid, epoch)
        writer.add_text('CNN Predictions', str(preds[:16].cpu().numpy()), epoch)
        writer.add_text('CNN Labels', str(labels[:16].cpu().numpy()), epoch)
writer.close()


CNN Epoch 1: Test accuracy = 0.8670
CNN Epoch 2: Test accuracy = 0.8835
CNN Epoch 3: Test accuracy = 0.9017
CNN Epoch 4: Test accuracy = 0.9060
CNN Epoch 5: Test accuracy = 0.9116
CNN Epoch 6: Test accuracy = 0.9120
CNN Epoch 7: Test accuracy = 0.9189
CNN Epoch 8: Test accuracy = 0.9147
CNN Epoch 9: Test accuracy = 0.9186
CNN Epoch 10: Test accuracy = 0.9204


In [ ]:
# 4. Sauvegarde des modèles
torch.save(mlp.state_dict(), "fashionmlp.pth")
torch.save(cnn.state_dict(), "fashioncnn.pth")

In [ ]:
# 5. Export ONNX
mlp.eval()
cnn.eval()
dummy = torch.randn(1, 1, 28, 28, device='cpu')
torch.onnx.export(mlp, dummy, "fashionmlp.onnx", input_names=["input"], output_names=["output"], opset_version=13)
torch.onnx.export(cnn, dummy, "fashioncnn.onnx", input_names=["input"], output_names=["output"], opset_version=13)